In [2]:
import gc
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, BatchNormalization, TimeDistributed, Conv1D, MaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import to_categorical
import random
from scipy.signal import savgol_filter
from tensorflow.keras.layers import Input, Masking, Dense, Dropout, GlobalAveragePooling1D, Layer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models , regularizers
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from keras.layers import Lambda
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict, Optional, Union, Callable
import copy
import json 

import os
import gc
import cv2
import json
import time
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
# import mediapipe as mp
# import tensorflow as tf
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output, FileLink

2025-06-28 20:01:19.978551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751140880.171861      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751140880.229777      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# pip install tensorflow==2.15.0 tensorflow-addons==0.22.0 --upgrade --force-reinstall
# pip install tensorflow
# !pip install -q git+https://github.com/tensorflow/addons.git


In [3]:
gc.collect()
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import os
os.listdir('/kaggle/input/new-landmarks')

['landmarks (1).npz', 'split_list.json']

In [5]:
landmarks2_dict = np.load("/kaggle/input/new-landmarks/landmarks (1).npz", allow_pickle=True)


In [6]:
# Load JSON file from /kaggle/working
with open("/kaggle/input/new-landmarks/split_list.json") as f:
    split_list = json.load(f)

# print(split_list)

In [7]:
X_train2 = []
Y_train2 = []

for key, arr in landmarks2_dict.items():
    # Extract word from key (e.g., 'RIGHT' from '6898343982713611-RIGHT 3')
    word = key.split('-')[1].split()[0].lower()

    X_train2.append(arr)
    Y_train2.append(word)


In [8]:

xx = ['accident', 'africa', 'all', 'apple', 'basketball', 'bed', 'before', 'bird', 'birthday', 'black', 'blue', 'book', 'bowling', 'brown', 'but', 'can', 'candy', 'chair', 'change', 'city', 'clothes', 'color', 'computer', 'cool', 'corn', 'cousin', 'cow', 'dance', 'dark', 'deaf', 'doctor', 'dog', 'drink', 'eat', 'enjoy', 'family', 'fine', 'finish', 'fish', 'forget', 'full', 'give', 'go', 'graduate', 'hat', 'hearing', 'help', 'hot', 'how', 'jacket', 'kiss', 'language', 'last', 'later', 'letter', 'like', 'man', 'many', 'meet', 'mother', 'need', 'no', 'now', 'orange', 'paint', 'paper', 'pink', 'pizza', 'play', 'pull', 'purple', 'right', 'same', 'school', 'secretary', 'shirt', 'short', 'study', 'table', 'tall', 'tell', 'thanksgiving', 'thin', 'thursday', 'time', 'visit', 'wait', 'walk', 'want', 'water', 'what', 'white', 'who', 'wife', 'woman', 'work', 'wrong', 'year', 'yellow', 'yes']

In [9]:
len(xx)

100

In [10]:
len(X_train2) , len(np.unique(Y_train2))

(4236, 195)

In [11]:
X_train22 = []
Y_train22 = []
split_list22 = []

for x, y, z in zip(X_train2, Y_train2 , split_list):
    if y in xx:
        split_list22.append(z)
        X_train22.append(x)
        Y_train22.append(y)


In [12]:
len(X_train22) , len(np.unique(Y_train22))

(4099, 98)

In [13]:
from collections import Counter

word_counts = Counter(split_list22)

print(word_counts)

Counter({'train': 1966, 'test': 1631, 'val': 502})


In [14]:
X_train_ = []
Y_train_ = []

X_val_ = []
Y_val_ = []

X_test_ = []
Y_test_ = []
for x, y, z in zip(X_train22, Y_train22 , split_list22):
    if z == 'train':
        X_train_.append(x)
        Y_train_.append(y)
    elif z == 'val':
        X_val_.append(x)
        Y_val_.append(y)
    else:
        X_test_.append(x)
        Y_test_.append(y)


In [15]:
landmarks_dict = np.load('/kaggle/input/mutemotion-output/landmarks_V3.npz', allow_pickle=True)
labels_dict = np.load('/kaggle/input/mutemotion-output/labels.npz', allow_pickle=True)
with open('/kaggle/input/mutemotion-output/WLASL_parsed_data.json', 'r') as json_file:
    data = json.load(json_file)
labels = []
with open('/kaggle/input/mutemotion-output/filtered_labels.txt', 'r') as file:
    labels = file.read().splitlines()

In [16]:
filtered_hand = list(range(21))

filtered_pose = [11, 12, 13, 14, 15, 16]

filtered_face = [4, 6, 8, 9, 33, 37, 40, 46, 52, 55, 61, 70, 80, 82, 84,
                 87, 88, 91, 105, 107, 133, 145, 154, 157, 159, 161, 163,
                 263, 267, 270, 276, 282, 285, 291, 300, 310, 312, 314, 317,
                 318, 321, 334, 336, 362, 374, 381, 384, 386, 388, 390, 468, 473]

HAND_NUM = len(filtered_hand)
POSE_NUM = len(filtered_pose)
FACE_NUM = len(filtered_face)

In [17]:
!pip install -q mediapipe==0.10.7


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 54.1 MB/s eta 0:00:00:00:0100:01


In [18]:
import mediapipe as mp
import tensorflow as tf

In [19]:
landmarks = (
    [x for x in filtered_hand] +
    [x + HAND_NUM for x in filtered_hand] +
    [x + HAND_NUM * 2 for x in filtered_pose] +
    [x + HAND_NUM * 2 + POSE_NUM for x in filtered_face]
)

# print(landmarks)
print(f'\nTotal Number: {len(landmarks)}') ,FACE_NUM


Total Number: 100


(None, 52)

In [20]:
landmarks2_dict = np.load('/kaggle/input/new-keypoints2/landmarkss (1).npz', allow_pickle=True)
# Load JSON file from /kaggle/working
with open("/kaggle/input/new-keypoints2/split_list (1).json") as f:
    split_list3 = json.load(f)



In [21]:
X_train3 = []
Y_train3 = []

for key, arr in landmarks2_dict.items():
    # Extract word from key (e.g., 'RIGHT' from '6898343982713611-RIGHT 3')
    word = key.split('-')[1].split()[0].lower()

    X_train3.append(arr)
    Y_train3.append(word)


In [22]:
len(X_train3) , len(np.unique(Y_train3))

(7308, 388)

In [23]:
xx = ['hair', 'headache', 'different', 'with', 'cat', 'red', 'knife', 'why', 'full', 'letter', 'right', 'write', 'have', 'sick', 'africa', 'same', 'but', 'door', 'yesterday', 'tea', 'show', 'teacher', 'run', 'cry', 'brother', 'week', 'birthday', 'cheat', 'read', 'movie', 'learn', 'rabbit', 'copy', 'name', 'trade', 'backpack', 'list', 'bad', 'sunday', 'happy', 'college', 'first', 'room', 'blanket', 'bathroom', 'example', 'daughter', 'window', 'late', 'take', 'buy', 'bar', 'travel', 'call', 'tired', 'you', 'again', 'share', 'coffee', 'inform', 'old', 'person', 'problem', 'friend', 'laugh', 'check', 'test', 'know', 'police', 'cute', 'ball', 'cold', 'home', 'lose', 'good', 'remember', 'stay', 'convince', 'west', 'president', 'theory', 'hour', 'light', 'pants', 'niece', 'nice', 'noon', 'some', 'past', 'party', 'center', 'improve', 'war', 'draw', 'hope', 'father', 'feel', 'house', 'heart', 'temperature', 'delay', 'secret', 'day', 'move', 'cough', 'jump', 'future', 'drop', 'silly', 'ugly', 'join', 'money', 'sleep', 'today', 'live', 'easy', 'nurse', 'wet', 'moon', 'plan', 'drive', 'business', 'milk', 'restaurant', 'far', 'practice', 'tomato', 'north', 'teach', 'saturday', 'delicious', 'girl', 'bake', 'crazy', 'month', 'careful', 'balance', 'mean', 'please', 'more', 'crash', 'husband', 'banana', 'make', 'russia', 'cheese', 'small', 'thank you', 'beard', 'most', 'your', 'flower', 'about', 'government', 'game', 'since', 'ride', 'clock', 'leave', 'because', 'win', 'office', 'tent', 'train', 'football', 'discuss', 'nephew', 'where', 'close', 'boy', 'fat', 'law', 'sad', 'soon', 'arrive', 'christmas', 'here', 'sign', 'sandwich', 'new', 'student', 'chat', 'approve', 'children', 'music', 'dress', 'salt', 'meat', 'near', 'country', 'order', 'pencil', 'none', 'hear', 'south', 'shoes', 'interest', 'traffic', 'throw', 'off']

In [24]:
X_train33 = []
Y_train33 = []
split_list33 = []

for x, y, z in zip(X_train3, Y_train3 , split_list3):
    if y in xx:
        split_list33.append(z)
        X_train33.append(x)
        Y_train33.append(y)


In [25]:
len(X_train33) , len(np.unique(Y_train33))

(7039, 193)

In [26]:
from collections import Counter

word_counts = Counter(split_list33)

print(word_counts)

Counter({'train': 3400, 'test': 2795, 'val': 844})


In [27]:
X_train__ = []
Y_train__ = []

X_val__ = []
Y_val__ = []

X_test__ = []
Y_test__ = []
for x, y, z in zip(X_train33, Y_train33 , split_list33):
    if z == 'train':
        X_train__.append(x)
        Y_train__.append(y)
    elif z == 'val':
        X_val__.append(x)
        Y_val__.append(y)
    else:
        X_test__.append(x)
        Y_test__.append(y)


In [28]:
def load_data(split, labels=None, max_labels=None, max_samples=None, landmarks=None, keys=None):
    
    if landmarks is None:
        landmarks = list(range(landmarks_dict['0'].shape[1]))
    
    if keys is None:
        keys = [k for k in landmarks_dict.keys()]
    
    if labels is not None:
        X = [landmarks_dict[k][:, landmarks, :] for k in keys
             if data[int(k)]['split'] == split and data[int(k)]['gloss'] in labels]
        Y = [data[int(k)]['gloss'] for k in keys
             if data[int(k)]['split'] == split and data[int(k)]['gloss'] in labels]
    
    elif max_samples is not None:
        X = [landmarks_dict[k][:, landmarks, :] for k in keys
             if data[int(k)]['split'] == split][:max_samples]
        Y = [data[int(k)]['gloss'] for k in keys
             if data[int(k)]['split'] == split][:max_samples]
    
    elif max_labels is not None:
        label_counts = {}
        for k in keys:
            label = data[int(k)]['gloss']
            label_counts[label] = label_counts.get(label, 0) + 1
        
        top_labels = sorted(label_counts, key=label_counts.get, reverse=True)[:max_labels]
        X = [landmarks_dict[k][:, landmarks, :] for k in keys
             if data[int(k)]['gloss'] in top_labels and data[int(k)]['split'] == split]
        Y = [data[int(k)]['gloss'] for k in keys
             if data[int(k)]['gloss'] in top_labels and data[int(k)]['split'] == split]
        
    else:
        X = [landmarks_dict[k][:, landmarks, :] for k in keys
             if data[int(k)]['split'] == split]
        Y = [data[int(k)]['gloss'] for k in keys
             if data[int(k)]['split'] == split]
    
    return X, Y

In [29]:
len(X_train_), len(X_val_), len(X_test_)

(1966, 502, 1631)

In [30]:
len(X_train__), len(X_val__), len(X_test__)

(3400, 844, 2795)

In [31]:
# X_train = X_train_ + X_test_ + X_train__ + X_test__
# Y_train = Y_train_ + Y_test_ + Y_train__ + Y_test__
# X_val=X_val_ + X_val__
# Y_val=Y_val_ + Y_val__
# X_test=X_test_ + X_test__ 
# Y_test=Y_test_ + Y_test__

In [32]:
X_train = X_train__ + X_test__
Y_train = Y_train__ + Y_test__
X_val= X_val__
Y_val= Y_val__
X_test= X_test__ 
Y_test= Y_test__

In [33]:
len(X_train), len(X_val), len(X_test)


(6195, 844, 2795)

In [34]:
len(np.unique(Y_train)), len(np.unique(Y_val)), len(np.unique(Y_test))


(193, 185, 193)

In [35]:
np.all(np.in1d(np.unique(Y_val), np.unique(Y_train))),\
np.all(np.in1d(np.unique(Y_test), np.unique(Y_train)))

(True, True)

In [36]:
X_train[0].shape, X_val[0].shape, X_test[0].shape

((173, 100, 3), (76, 100, 3), (42, 100, 3))

In [37]:
def mirror_keypoints(sample):
    mirrored = sample.copy()
    
    # 1. Flip X-axis
    mirrored[:, 0] = 1.0 - mirrored[:, 0]

    # 2. Swap hands (use already-flipped values)
    left_hand = mirrored[:21].copy()
    right_hand = mirrored[21:42].copy()
    mirrored[:21], mirrored[21:42] = right_hand, left_hand

    # 3. Swap pose landmarks (already flipped)
    pose_l = [0, 2, 4]
    pose_r = [1, 3, 5]
    for l, r in zip(pose_l, pose_r):
        mirrored[42 + l], mirrored[42 + r] = mirrored[42 + r].copy(), mirrored[42 + l].copy()

    # 4. Swap symmetric face landmarks
    face_indices = [4, 6, 8, 9, 33, 37, 40, 46, 52, 55, 61, 70, 80, 82, 84,
                    87, 88, 91, 105, 107, 133, 145, 154, 157, 159, 161, 163,
                    263, 267, 270, 276, 282, 285, 291, 300, 310, 312, 314, 317,
                    318, 321, 334, 336, 362, 374, 381, 384, 386, 388, 390, 468, 473]

    face_mapping = {
        33: 263, 37: 267, 40: 270, 46: 276, 52: 282, 55: 285, 61: 291,
        70: 300, 80: 310, 82: 312, 84: 314, 87: 317, 88: 318, 91: 321,
        133: 362, 145: 374, 154: 381, 157: 384, 159: 386, 161: 388, 163: 390
    }

    face_idx_to_i = {v: i for i, v in enumerate(face_indices)}
    for k, v in face_mapping.items():
        if k in face_idx_to_i and v in face_idx_to_i:
            i, j = face_idx_to_i[k] + 48, face_idx_to_i[v] + 48  # +48 offset for face
            mirrored[i], mirrored[j] = mirrored[j].copy(), mirrored[i].copy()

    return mirrored

def allframes(data):
    pp = []
    for x in data:
        pp.append(mirror_keypoints(x))
    pp = np.array(pp)
    return pp

def allsampels(data):
    final = []
    for x in data:
        final.append(allframes(x))
    return final
    

In [38]:
X_train_mirror = allsampels(X_train)
Y_train_mirror = Y_train


In [39]:
len(X_train_mirror)

6195

In [40]:
def rotate(data, rotation_matrix):
    frames, landmarks, _ = data.shape
    center = np.array([0.5, 0.5, 0])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data = data.reshape(-1, 3)
    data[non_zero] -= center
    data[non_zero] = np.dot(data[non_zero], rotation_matrix.T)
    data[non_zero] += center
    data = data.reshape(frames, landmarks, 3)
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def rotate_z(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    return rotate(data, rotation_matrix)

def rotate_y(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

def rotate_x(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

def zoom(data):
    factor = np.random.uniform(0.8, 1.2)
    center = np.array([0.5, 0.5])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], :2] = (
        (data[non_zero[:, 0], non_zero[:, 1], :2] - center) * factor + center
    )
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def shift(data):
    x_shift = np.random.uniform(-0.2, 0.2)
    y_shift = np.random.uniform(-0.2, 0.2)
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], 0] += x_shift
    data[non_zero[:, 0], non_zero[:, 1], 1] += y_shift
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def mask(data):
    frames, landmarks, _ = data.shape
    num_hands = int(0.3 * 42)
    num_rest = int(0.6 * (landmarks - 42))

    mask = np.zeros(landmarks, dtype=bool)
    indices = np.concatenate([
        np.random.choice(42, num_hands, replace=False),
        np.random.choice(landmarks - 42, num_rest, replace=False) + 42
    ])
    mask[indices] = True
    data[:, mask] = 0
    return data

def speedup(data):
    return data[::2]


def apply_augmentations(data):
    aug_functions = [rotate_x, rotate_y, rotate_z, zoom, shift, mask, speedup]
    np.random.shuffle(aug_functions)
    counter = 0
    for fun in aug_functions:
        if np.random.rand() < 0.5:
            data = fun(data)
            counter += 1
    
    if counter == 0:
        data = apply_augmentations(data)

    return data


def augment(X, Y, num=None):
    X_aug = X.copy()
    Y_aug = Y.copy()
    
    if num == None:
        for i in tqdm(range(len(Y)), ncols=100):
            num_aug = np.random.choice([1, 2, 3])
            for n in range(num_aug):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])
    elif num > 0:
        for i in tqdm(range(len(Y)), ncols=100):
            for n in range(num):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])

    return X_aug, Y_aug




In [41]:
X_aug, Y_aug = augment(X_train, Y_train, num=1)
print(f"Original data: {len(X_train)} samples")
print(f"Augmented data: {len(X_aug)} samples")

100%|██████████████████████████████████████████████████████████| 6195/6195 [00:30<00:00, 201.21it/s]

Original data: 6195 samples
Augmented data: 12390 samples


In [42]:
X_aug = X_aug + X_train_mirror
Y_aug = Y_aug + Y_train_mirror

In [43]:
len(X_aug)

18585

In [44]:
permutation_train = list(range(len(Y_aug)))
np.random.shuffle(permutation_train)
X_aug = [X_aug[i] for i in permutation_train]
Y_aug = [Y_aug[i] for i in permutation_train]

permutation_val = list(range(len(Y_val)))
np.random.shuffle(permutation_val)
X_val = [X_val[i] for i in permutation_val]
Y_val = [Y_val[i] for i in permutation_val]

permutation_test = list(range(len(Y_test)))
np.random.shuffle(permutation_test)
X_test = [X_test[i] for i in permutation_test]
Y_test = [Y_test[i] for i in permutation_test]

In [45]:
# Function to determine the best sequence length
def determine_sequence_length(sequences):
    """
    Analyzes the distribution of sequence lengths to find a good max_frames value
    """
    lengths = [seq.shape[0] for seq in sequences]
    
    # Calculate statistics
    mean_length = np.mean(lengths)
    median_length = np.median(lengths)
    p95_length = np.percentile(lengths, 95)
    return int(p95_length)


In [46]:
# Assuming your list is called `X_train`
max_frames = determine_sequence_length(X_aug)
# max_frames = 141

print("Maximum number of frames (shape[0]):", max_frames)

Maximum number of frames (shape[0]): 141


In [47]:
def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    
    X_padded = []
    for x in X:
        if len(x) > length:
            X_padded.append(x[:length]) #truncate
        else:
            pad_length = length - len(x)
            X_padded.append(np.pad(
                x, ((0, pad_length), (0, 0), (0, 0)),
                mode='constant', constant_values=pad
            ))
            
    X_padded = np.array(X_padded)
#     Y = np.array(Y)
    return X_padded, Y

In [48]:
X_train_processed, y_train_processed = padding(X_aug, Y_aug, length=max_frames, pad=0)
X_val_processed, y_val_processed = padding(X_val, Y_val, length=max_frames, pad=0)
X_test_processed, y_test_processed = padding(X_test, Y_test, length=max_frames, pad=0)

In [49]:
# y_train_processed = Y_aug
# y_val_processed = Y_val
# y_test_processed = Y_test

In [50]:
len(X_train_processed), len(X_train_processed[0]), len(X_train_processed[0][0]), len(X_train_processed[0][0][0])


(18585, 141, 100, 3)

In [51]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Y_train = np.array([labels_dict[label] for label in Y_train])
# Y_val = np.array([labels_dict[label] for label in Y_val])
# Y_test = np.array([labels_dict[label] for label in Y_test])
# Fit the encoder on training labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_processed)
y_val_encoded = label_encoder.transform(y_val_processed)
y_test_encoded = label_encoder.transform(y_test_processed)

In [52]:
import pickle

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("Label encoder saved successfully.")

Label encoder saved successfully.


In [53]:
num_classes = len(label_encoder.classes_)
num_classes

193

In [54]:
gc.collect()
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [55]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

class PaddedDataGenerator(Sequence):
    def __init__(self, X_all, y_all, batch_size=64, max_len=max_frames, pad_value=0.0, shuffle=True):
        self.X_all = X_all  # list of arrays with shape (T, 180, 3)
        self.y_all = y_all
        self.batch_size = batch_size
        self.max_len = max_len
        self.pad_value = pad_value
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.X_all))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.X_all) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = [self.X_all[i] for i in batch_indices]
        y_batch = [self.y_all[i] for i in batch_indices]

        # Apply padding manually
        X_padded , _ = padding(X_batch, y_batch, length=max_frames, pad=0)
        y_batch = np.array(y_batch)

        return X_padded, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def pad_sequence(self, x, max_len, pad_value):
        current_len = x.shape[0]
        if current_len >= max_len:
            return x[:max_len]
        pad_width = ((0, max_len - current_len), (0, 0), (0, 0))
        return np.pad(x, pad_width, mode='constant', constant_values=pad_value)


In [56]:
train_gen = PaddedDataGenerator(X_aug, y_train_encoded, batch_size=64, max_len=max_frames)
val_gen = PaddedDataGenerator(X_val, y_val_encoded, batch_size=64, max_len=max_frames)


In [57]:
import tensorflow as tf

# ✅ Keep ECA block — uses only Conv1D and pooling, so it’s fine.
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.kernel_size = kernel_size
        self.gap = tf.keras.layers.GlobalAveragePooling1D()  # ✅ create once!
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs):
        nn = self.gap(inputs)  # ✅ reuse
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:, None, :]
        return inputs * nn


class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, kernel_size=17, dilation_rate=1, use_bias=False, name='', **kwargs):
        super().__init__(name=name, **kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate * (kernel_size - 1), 0), name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
            kernel_size,
            strides=1,
            dilation_rate=dilation_rate,
            padding='valid',
            use_bias=use_bias,
            name=name + '_dwconv'
        )

    def call(self, inputs):
        x = self.causal_pad(inputs)
        return self.dw_conv(x)

def Conv1DBlock(channel_size, kernel_size, dilation_rate=1, drop_rate=0.0, expand_ratio=2, name=None):
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(channels_expand, use_bias=True, activation='swish', name=name + '_expand_conv')(inputs)
        x = CausalDWConv1D(kernel_size, dilation_rate=dilation_rate, name=name + '_dwconv')(x)
        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)
        x = ECA()(x)
        x = tf.keras.layers.Dense(channel_size, use_bias=True, name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None, 1, 1), name=name + '_drop')(x)

        if channels_in == channel_size:
            x = tf.keras.layers.Add(name=name + '_add')([x, skip])
        return x
    return apply

# ✅ Replace TransformerBlock with simple Dense mixing block
def SimpleTransformerLikeBlock(dim, expand=4, drop_rate=0.2):
    def apply(inputs):
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(inputs)
        x = tf.keras.layers.Dense(dim * expand, activation='swish')(x)
        x = tf.keras.layers.Dense(dim)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None, 1, 1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        return x
    return apply

def get_model(num_classes, max_frames=120, dim=192):
    inp = tf.keras.layers.Input((max_frames, 100, 3), name='input')
    x = tf.keras.layers.Reshape((max_frames, 300))(inp)

    ksize = 17
    x = tf.keras.layers.Dense(dim, use_bias=False, name='stem_conv')(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.95, name='stem_bn')(x)

    # Conv blocks
    x = Conv1DBlock(dim, ksize, drop_rate=0.2, name='conv1')(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.2, name='conv2')(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.2, name='conv3')(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.2, name='conv4')(x)

    # Transformer replacement
    x = SimpleTransformerLikeBlock(dim, expand=2)(x)

    x = Conv1DBlock(dim, ksize, drop_rate=0.3, name='conv5')(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.3, name='conv6')(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.3, name='conv7')(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.3, name='conv8')(x)

    x = SimpleTransformerLikeBlock(dim, expand=2)(x)

    x = tf.keras.layers.Dense(dim * 2, activation=None, name='top_conv')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Plain dropout, works for inference
    out = tf.keras.layers.Dense(num_classes, name='classifier')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    return model


In [58]:
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr_scheduler
)

I0000 00:00:1751140995.614704      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [59]:
# num_classes = 193  # ✅ CHANGE TO YOUR VALUE
max_frames = 143   # ✅ YOUR max_frames

model = get_model(num_classes=num_classes, max_frames=max_frames)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)        │ (None, 143, 100, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 143, 300)       │              0 │ input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Dense)         │ (None, 143, 192)       │         57,600 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 143, 192)       │            768 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_expand_conv (Dense) │ (None, 143, 384)       │         74,112 │ stem_bn[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_dwconv              │ (None, 143, 384)       │          6,528 │ conv1_expand_conv[0][… │
│ (CausalDWConv1D)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 143, 384)       │          1,536 │ conv1_dwconv[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ eca (ECA)                 │ (None, 143, 384)       │              5 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_project_conv        │ (None, 143, 192)       │         73,920 │ eca[0][0]              │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_drop (Dropout)      │ (None, 143, 192)       │              0 │ conv1_project_conv[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_add (Add)           │ (None, 143, 192)       │              0 │ conv1_drop[0][0],      │
│                           │                        │                │ stem_bn[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_expand_conv (Dense) │ (None, 143, 384)       │         74,112 │ conv1_add[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_dwconv              │ (None, 143, 384)       │          6,528 │ conv2_expand_conv[0][… │
│ (CausalDWConv1D)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_bn                  │ (None, 143, 384)       │          1,536 │ conv2_dwconv[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ eca_1 (ECA)               │ (None, 143, 384)       │              5 │ conv2_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_project_conv        │ (None, 143, 192)       │         73,920 │ eca_1[0][0]            │
│ (Dense)              

 Total params: 1,753,193 (6.69 MB)

 Trainable params: 1,745,897 (6.66 MB)

 Non-trainable params: 7,296 (28.50 KB)

In [64]:
acc_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_accuracy_.weights.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

loss_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_loss_.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=50,
    restore_best_weights=True
)

In [65]:
model = get_model(num_classes=num_classes, max_frames=max_frames)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics=['accuracy']
)


In [66]:
# try:
#     hist = model.fit(
#         X_train_processed, y_train_encoded,
#         validation_data=(X_val_processed, y_val_encoded),
#         epochs=1000,
#         batch_size=32,
#         callbacks=[acc_checkpoint, loss_checkpoint, early_stopping]
#     )
    
# except KeyboardInterrupt:
#     model.load_weights('/kaggle/input/best-accuracy-weights-h5/best_accuracy_weights.h5')
#     print("\nManual interruption detected. Training stopped.")


In [70]:
try:
    hist = model.fit(
        train_gen,  # ✅
        validation_data=val_gen,  # ✅
        epochs=1000,
        callbacks=[acc_checkpoint, loss_checkpoint, early_stopping]
    )
except KeyboardInterrupt:
    model.load_weights('/kaggle/input/best-accuracy-weights-h5/best_accuracy_weights.h5')
    print("\nManual interruption detected. Training stopped.")


Epoch 1/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 19s 64ms/step - accuracy: 0.9387 - loss: 0.2473 - val_accuracy: 0.8246 - val_loss: 1.5216
Epoch 2/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9414 - loss: 0.2287 - val_accuracy: 0.8223 - val_loss: 1.6572
Epoch 3/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 19s 64ms/step - accuracy: 0.9399 - loss: 0.2350 - val_accuracy: 0.8555 - val_loss: 1.4036
Epoch 4/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 19s 63ms/step - accuracy: 0.9509 - loss: 0.1863 - val_accuracy: 0.8140 - val_loss: 1.6911
Epoch 5/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9433 - loss: 0.2133 - val_accuracy: 0.8377 - val_loss: 1.3563
Epoch 6/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9474 - loss: 0.2192 - val_accuracy: 0.7950 - val_loss: 1.8791
Epoch 7/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9381 - loss: 0.2614 - val_accuracy: 0.8246 - val_loss: 1.4754
Epoch 8/1000
291/291 ━━━━━━━━━━━━━━━━━━━━ 18s 63ms/step - accuracy: 0.9444 -

In [71]:
print("done")

done


In [72]:
model.save('sign_language_model.h5')

In [73]:
import tensorflow as tf

# Load your Keras model
model = tf.keras.models.load_model('sign_language_model.h5', compile=False)

# Create converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: for smaller size
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert
tflite_model = converter.convert()

# Save .tflite
with open('sign_language_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite model saved!")


ValueError: Unknown layer: 'CausalDWConv1D'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [74]:
model = tf.keras.models.load_model(
    'sign_language_model.h5',
    custom_objects={
        'CausalDWConv1D': CausalDWConv1D,
        'ECA': ECA
    }
)

In [75]:
import tensorflow as tf

# Load your Keras model
model = tf.keras.models.load_model('sign_language_model.h5', compile=False)

# Create converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optional: for smaller size
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert
tflite_model = converter.convert()

# Save .tflite
with open('sign_language_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite model saved!")


ValueError: Unknown layer: 'CausalDWConv1D'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [76]:
# Just use your existing `model` object
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save it
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpdr5iqi3q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 143, 100, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 193), dtype=tf.float32, name=None)
Captures:
  139319147730064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147725840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147726800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147729680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147735440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147735056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147727568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147727376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147727760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147725456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147732752: Te

W0000 00:00:1751144103.408899      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1751144103.408950      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1751144103.466669      35 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


In [77]:
import tensorflow as tf

# If you have a SavedModel dir (like /tmp/tmphkai2n3s)
converter = tf.lite.TFLiteConverter.from_saved_model('/tmp/tmphkai2n3s')
tflite_model = converter.convert()

# Save the .tflite file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite model saved as model.tflite")


OSError: SavedModel file does not exist at: /tmp/tmphkai2n3s/{saved_model.pbtxt|saved_model.pb}

In [78]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite model saved as model.tflite")


Saved artifact at '/tmp/tmpo8k04_ik'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 143, 100, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 193), dtype=tf.float32, name=None)
Captures:
  139319147730064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147725840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147726800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147729680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147735440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147735056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147727568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147727376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147727760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147725456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139319147732752: Te

W0000 00:00:1751144115.037073      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1751144115.037104      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.


In [79]:
!ls -lh model.tflite


-rw-r--r-- 1 root root 6.7M Jun 28 20:55 model.tflite


In [79]:
from IPython.display import FileLink
FileLink('model.tflite')


/kaggle/working/model.tflite

In [80]:
import tensorflow as tf
import numpy as np

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)


Input details: [{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1, 143, 100,   3], dtype=int32), 'shape_signature': array([ -1, 143, 100,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 240, 'shape': array([  1, 193], dtype=int32), 'shape_signature': array([ -1, 193], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [81]:
# Create dummy input data (batch size 1)
dummy_input = np.random.rand(1, 143, 100, 3).astype(np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], dummy_input)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Output shape:", output_data.shape)
print("Output:", output_data)


Output shape: (1, 193)
Output: [[  31.22481      31.383783     46.254585    -10.375814    420.63303
   153.47751    -184.56134     -96.39678     -88.592384    -36.701023
   -91.864235     15.238624    221.6397       18.74339     110.20595
   146.45847      48.595917    120.33687      19.277023    -57.890686
    25.33636     -94.466774    -74.38496     -11.420822   -141.34982
    75.76979      79.20773    -132.72598    -151.97954     -74.247444
   246.89494      60.954853     14.442897   -128.85228      63.225945
  -171.22168      -4.132922   -159.95816      30.322384    -49.239674
    28.63538     122.29365      31.241768    -47.398388    177.87799
   330.84943      12.181226     41.526802     42.88259     -43.003933
  -133.54236     226.72641     173.859       167.52516    -141.1
   -66.844       -46.259594    -27.500416    194.15958     -33.56527
    84.1169      -43.11155     129.09297     151.79608      56.81295
   219.08403      73.98317     119.34032      -0.48903346  154.2384
  

In [81]:
len(X_val_processed)

844

In [85]:
import tensorflow as tf
import numpy as np

# ✅ Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/kaggle/working/model.tflite')
interpreter.allocate_tensors()

# ✅ Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ✅ For sanity check
print("Input shape expected:", input_details[0]['shape'])
print("Input dtype expected:", input_details[0]['dtype'])

# ✅ Make sure your data shape matches
print("Sample val shape:", X_val_processed2[0].shape)

# ✅ Loop over val data
correct = 0
total = len(X_val_processed2)

for i in range(total):
    # Add batch dim & cast to float32
    input_data = np.expand_dims(X_val_processed2[i], axis=0).astype(np.float32)
    
    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Inference
    interpreter.invoke()
    
    # Get prediction
    output = interpreter.get_tensor(output_details[0]['index'])
    pred_label = np.argmax(output)
    true_label = y_val_encoded[i]   # ⚠️ Must be numeric encoded

    if pred_label == true_label:
        correct += 1

accuracy = correct / total

print(f"TFLite validation accuracy: {accuracy:.4f}")


Input shape expected: [  1 143 100   3]
Input dtype expected: <class 'numpy.float32'>
Sample val shape: (143, 100, 3)
TFLite validation accuracy: 0.8578


In [84]:
# Re-pad to match the TFLite input shape
def pad_sequences(X, target_len):
    X_padded = []
    for x in X:
        if len(x) >= target_len:
            X_padded.append(x[:target_len])
        else:
            pad_len = target_len - len(x)
            X_padded.append(
                np.pad(x, ((0, pad_len), (0, 0), (0, 0)), mode='constant', constant_values=0)
            )
    return np.array(X_padded)

# Re-pad your val set
X_val_processed2 = pad_sequences(X_val_processed, target_len=143)

print(X_val_processed.shape)


(844, 141, 100, 3)


In [115]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder  # Assuming you use this for inverse_transform

# ======= CONFIG =======
TFLITE_MODEL_PATH = "/kaggle/working/model.tflite"  # <<< CHANGE THIS
VIDEOS_DIR = "/kaggle/input/qwsdvb/New folder"  # <<< your videos folder
PAD_LENGTH = 143  # must match your TFLite model input
# SAMPLE_INDEX = 2  # which processed video to test
# ======================

# === Mediapipe configs ===
filtered_hand = list(range(21))
filtered_pose = [11, 12, 13, 14, 15, 16]
filtered_face = [4, 6, 8, 9, 33, 37, 40, 46, 52, 55, 61, 70, 80, 82, 84,
                 87, 88, 91, 105, 107, 133, 145, 154, 157, 159, 161, 163,
                 263, 267, 270, 276, 282, 285, 291, 300, 310, 312, 314, 317,
                 318, 321, 334, 336, 362, 374, 381, 384, 386, 388, 390, 468, 473]

HAND_NUM = len(filtered_hand)
POSE_NUM = len(filtered_pose)
FACE_NUM = len(filtered_face)

def get_frame_landmarks(frame, hands, pose, face_mesh):
    all_landmarks = np.zeros((HAND_NUM * 2 + POSE_NUM + FACE_NUM, 3))

    results_hands = hands.process(frame)
    if results_hands.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            if results_hands.multi_handedness[i].classification[0].index == 0:
                all_landmarks[:HAND_NUM, :] = np.array(
                    [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
            else:
                all_landmarks[HAND_NUM:HAND_NUM * 2, :] = np.array(
                    [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])

    results_pose = pose.process(frame)
    if results_pose.pose_landmarks:
        all_landmarks[HAND_NUM * 2:HAND_NUM * 2 + POSE_NUM, :] = np.array(
            [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])[filtered_pose]

    results_face = face_mesh.process(frame)
    if results_face.multi_face_landmarks:
        all_landmarks[HAND_NUM * 2 + POSE_NUM:, :] = np.array(
            [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark])[filtered_face]

    return all_landmarks

def get_video_landmarks(video_path):
    hands = mp.solutions.hands.Hands()
    pose = mp.solutions.pose.Pose()
    face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

    cap = cv2.VideoCapture(video_path)
    all_frame_landmarks = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_landmarks = get_frame_landmarks(frame, hands, pose, face_mesh)
        all_frame_landmarks.append(frame_landmarks)

    cap.release()
    hands.close()
    pose.close()
    face_mesh.close()

    return np.array(all_frame_landmarks)

def process_all_videos(videos_dir):
    all_landmarks = []
    labels = []

    for filename in os.listdir(videos_dir):
        if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            video_path = os.path.join(videos_dir, filename)
            print(f"Processing: {filename}")
            landmarks = get_video_landmarks(video_path)
            all_landmarks.append(landmarks)
            labels.append(os.path.splitext(filename)[0])
    return all_landmarks, labels

def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    X_padded = []
    for x in X:
        if len(x) > length:
            X_padded.append(x[:length])
        else:
            pad_length = length - len(x)
            X_padded.append(np.pad(x, ((0, pad_length), (0, 0), (0, 0)), mode='constant', constant_values=pad))
    return np.array(X_padded), Y

# === Process videos ===
video_data, video_labels = process_all_videos(VIDEOS_DIR)
X_padded, Y_padded = padding(video_data, video_labels, length=PAD_LENGTH)

# === Load your label encoder ===
# This assumes you saved your LabelEncoder to encoder.pkl
import joblib
encoder = joblib.load("/kaggle/working/label_encoder.pkl")  # <<< adjust path if needed

# === Load TFLite ===
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# === Run inference for one sample ===



Processing: girl.mp4
Processing: myDaughter .mp4
Processing: myfather.mp4


In [121]:
SAMPLE_INDEX = 2

sample = np.expand_dims(X_padded[SAMPLE_INDEX], axis=0).astype(np.float32)

print("TFLite input shape:", input_details[0]['shape'])
print("Sample shape:", sample.shape)

interpreter.set_tensor(input_details[0]['index'], sample)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

pred_probs = output_data[0]
top10_indices = np.argsort(pred_probs)[-10:][::-1]
top10_labels = encoder.inverse_transform(top10_indices)
top10_probs = pred_probs[top10_indices]

print("\n✅ Top 10 Predictions:")
for i in range(10):
    print(f"{i+1}. {top10_labels[i]}: {top10_probs[i]:.4f}")

print(f"\nTrue Label: {Y_padded[SAMPLE_INDEX]}")

TFLite input shape: [  1 143 100   3]
Sample shape: (1, 143, 100, 3)

✅ Top 10 Predictions:
1. hair: 43.0918
2. noon: 31.3556
3. boy: 29.5759
4. nephew: 28.4305
5. know: 25.7045
6. remember: 25.5284
7. father: 25.5126
8. sick: 22.3254
9. balance: 20.3300
10. teach: 19.6913

True Label: myfather


In [103]:
print(sorted(set(y_train_processed)))

['about', 'africa', 'again', 'arrive', 'backpack', 'bad', 'bake', 'balance', 'ball', 'banana', 'bar', 'bathroom', 'beard', 'because', 'birthday', 'blanket', 'boy', 'brother', 'business', 'but', 'buy', 'call', 'careful', 'cat', 'center', 'chat', 'cheat', 'check', 'cheese', 'children', 'christmas', 'clock', 'close', 'coffee', 'cold', 'college', 'convince', 'copy', 'cough', 'country', 'crash', 'crazy', 'cry', 'cute', 'daughter', 'day', 'delicious', 'different', 'discuss', 'door', 'draw', 'dress', 'drive', 'drop', 'easy', 'example', 'far', 'fat', 'father', 'feel', 'first', 'flower', 'football', 'friend', 'full', 'future', 'game', 'girl', 'good', 'government', 'hair', 'happy', 'have', 'headache', 'hear', 'heart', 'here', 'home', 'hope', 'hour', 'house', 'husband', 'improve', 'interest', 'join', 'jump', 'knife', 'know', 'late', 'laugh', 'law', 'learn', 'leave', 'letter', 'light', 'list', 'live', 'lose', 'make', 'mean', 'meat', 'milk', 'money', 'month', 'moon', 'more', 'most', 'move', 'movie'